In [1]:
from playwright.async_api import async_playwright
import asyncio

In [2]:
async def fetch_with_playwright():
    news_list = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        
        page = await browser.new_page()
        await page.set_extra_http_headers({
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36"
        })
        await page.goto("https://www.prothomalo.com/collection/latest", timeout=60000)
        await page.wait_for_selector("#container", timeout=15000)
        #Wait for a card to appear inside container
        await page.wait_for_selector("#container div.news_with_item", timeout=10000)
        elements = await page.locator("h3.headline-title").all()
        for el in elements:
            link_el  =  el.locator("a.title-link")
            href = await link_el.get_attribute("href")
            title = (await el.inner_text()).strip()
            news_list.append((title, href))
            
        await browser.close()
    return news_list
        #for i in range(articles.count())
     

In [3]:
news_list = await fetch_with_playwright() 
#print(news_list)

In [4]:
from pymongo import MongoClient
from datetime import datetime
client = MongoClient("mongodb://localhost:27017/") # need to on env. for local can't do it right now.
db = client["bd-newspaper-collection"]

collection = db["articles"]

for title,url in news_list:
    if not collection.find_one({"url": url}):
        collection.insert_one({
            "title": title,
            "url": url,
            "media_name": "prothomalo",
            "scraped_at": datetime.now()
        })
        print(f"Inserted: {title}")
    else:
        print(f"Already Exits {title}")


Inserted: গণহত্যা বন্ধের দাবিতে নর্থ সাউথসহ চার বেসরকারি বিশ্ববিদ্যালয়ে বিক্ষোভ
Inserted: মেরিল ক্যাফে লাইভ প্রশংসিত হচ্ছে অপুর অভিনয়
Inserted: হত্যা মামলায় ৭ দিনের রিমান্ডে সাবেক এমপি আফজাল হোসেন
Inserted: বিপন্ন কাছিম রক্ষায় বন্ধ্যা করা হচ্ছে সেন্ট মার্টিন দ্বীপের ৩ হাজার কুকুরকে
Inserted: ইসরায়েলের বিরুদ্ধে ওআইসিকে পদক্ষেপ নিতে জামায়াতের আহ্বান
Already Exits আশিকুরের মৃত্যু ‘মাথায় গুলি নিয়াই ভাইটা মইরা গেল...চিকিৎসাও পাইল না’
Already Exits পাল্টা শুল্ক তিন মাস স্থগিতের অনুরোধ করে ডোনাল্ড ট্রাম্পকে চিঠি দিলেন প্রধান উপদেষ্টা
Already Exits ‘আমি নিতান্তই কোনো সংখ্যা হতে চাই না, আমি গাজার এক বাস্তব গল্প’
Already Exits মার্কিন শুল্কনীতির ধাক্কায় এশিয়া ও ইউরোপের শেয়ারবাজারে বড় ধস
Already Exits ফিলিস্তিনি ভূখণ্ড দখল করে যেভাবে গড়ে উঠল ইসরায়েল রাষ্ট্র
Inserted: সরাসরি সচিবালয়মুখী পদযাত্রায় পুলিশের হামলার প্রতিবাদে চাকরিচ্যুত বিডিআর সদস্যদের সংবাদ সম্মেলন
Already Exits ‘আমি নিতান্তই কোনো সংখ্যা হতে চাই না, আমি গাজার এক বাস্তব গল্প’
Already Exits মুঠোফোনের চার্জার, এয়ারপডে লুকিয়ে আনেন কোটির 

In [12]:
len(news_list)

25